On 2018, Kaggle launched a competition in association with Jigsaw/Conversation AI to classify toxic comments. The original competition can be viewed [here](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge). A toxic comment is a comment that is rude, disrespectful or otherwise likely to make someone leave a discussion. The goal of that competition was to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate.

However, the models developed in that competition unfortunately associated the targetted group with toxicity, i.e. "gay". For example, a comment like "I am a gay woman" would be classified as toxic. This happened as the examples of identities associated with toxicity outnumbered neutral comments regarding the same identity.

Therefore, the same team launched a new competition to recognize unintended bias towards identities. We are asked to use a dataset labeled with the associated identity. Let's look into the dataset and understand it first, so we can create a model that can better deal with bias.

This notebook is heavily based on the kernel:
https://www.kaggle.com/nz0722/simple-eda-text-preprocessing-jigsaw. The purpose of this Kernel is to walk Kaggle newbies through the process of data exploration and visualization. In this notebook, we will use Pandas to do a little bit of data wrangling and Plotly and Seaborn to visualize the result of our wrangling. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
print(os.listdir("../input"))

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

## Section 1: Understanding The Shape of Data

In [ ]:
# First, lets count how much data we have!
# Во-первых, давайте посчитаем, сколько данных у нас есть!
train_len, test_len = len(train_df.index), len(test_df.index)
print(f'train size: {train_len}, test size: {test_len}')

If you want to understand each of the field, it is a good idea to read the data section of the Kaggle competition 
Если вы хотите разобраться в каждой области, неплохо было бы прочитать раздел данных конкурса Kaggle(https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification/data). Let's take a couple of minutes to read this section first. Давайте сначала уделим пару минут чтению этого раздела.

In [ ]:
# also, lets take a quick look at what we have 
# также давайте взглянем на то, что у нас есть
train_df.head()

In [ ]:
# its always a good idea to count the amount of missing values before diving into any analysis
# Lets also see how many missing values (in percentage) we are dealing with
# всегда полезно посчитать количество пропущенных значений, прежде чем углубляться в любой анализ
# Давайте также посмотрим, сколько пропущенных значений (в процентах) мы имеем дело с
miss_val_train_df = train_df.isnull().sum(axis=0) / train_len
miss_val_train_df = miss_val_train_df[miss_val_train_df > 0] * 100
miss_val_train_df

> As you can see from the table above, a large portion of the data is missing the identity tag. However, as the number is the same for the tags, I assume that the data is complete for the part which has identity tags.
Как видно из таблицы выше, большая часть данных не имеет идентификационного тега. Тем не менее, так как число одинаково для тегов, я предполагаю, что данные полны для части, которая имеет идентификационные теги.

## Section 2: Identity Analysis and Barplots Идентификационный анализ и графики

Now that we know a large portion of our dataset is doesn't have the group identity, (for now) we can drop it before we do any basic analysis. First simple and interesting question to answer would be which identity appears the most in the dataset. 
Теперь, когда мы знаем, что большая часть нашего набора данных не имеет идентификатора группы, (на данный момент) мы можем отбросить его, прежде чем делать какой-либо базовый анализ. Первый простой и интересный вопрос, на который нужно ответить, - какая личность встречается чаще всего в наборе данных.

In [ ]:
# lets create a list of all the identities tagged in this dataset. This list given in the data section of this competition. 
#Давайте создадим список всех идентификаторов, помеченных в этом наборе данных. 
#Этот список приведен в разделе данных этого конкурса.
identities = ['male','female','transgender','other_gender','heterosexual','homosexual_gay_or_lesbian',
              'bisexual','other_sexual_orientation','christian','jewish','muslim','hindu','buddhist',
              'atheist','other_religion','black','white','asian','latino','other_race_or_ethnicity',
              'physical_disability','intellectual_or_learning_disability','psychiatric_or_mental_illness',
              'other_disability']

In [ ]:
# getting the dataframe with identities tagged получение фрейма данных с тэгами
train_labeled_df = train_df.loc[:, ['target'] + identities ].dropna()
# lets define toxicity as a comment with a score being equal or .5
# in that case we divide it into two dataframe so we can count toxic vs non toxic comment per identity
# позволяет определить токсичность как комментарий с оценкой равной или .5
# в этом случае мы делим его на два кадра данных, чтобы мы могли посчитать токсичные и нетоксичные 
# комментарии по каждому идентификатору
toxic_df = train_labeled_df[train_labeled_df['target'] >= .5][identities]
non_toxic_df = train_labeled_df[train_labeled_df['target'] < .5][identities]

In [ ]:
# at first, we just want to consider the identity tags in binary format. 
# So if the tag is any value other than 0 we consider it as 1.
# во-первых, мы просто хотим рассмотреть теги идентичности в двоичном формате. 
# Поэтому, если тегом является любое значение, отличное от 0, мы рассматриваем его как 1.
toxic_count = toxic_df.where(train_labeled_df == 0, other = 1).sum()
non_toxic_count = non_toxic_df.where(train_labeled_df == 0, other = 1).sum()

In [ ]:
# now we can concat the two series together to get a toxic count vs non toxic count for each identity
# теперь мы можем объединить две серии вместе, 
# чтобы получить подсчет токсичности против нетоксичности для каждой идентичности
toxic_vs_non_toxic = pd.concat([toxic_count, non_toxic_count], axis=1)
toxic_vs_non_toxic = toxic_vs_non_toxic.rename(index=str, columns={1: "non-toxic", 0: "toxic"})
# here we plot the stacked graph but we sort it by toxic comments to (perhaps) see something interesting
# здесь мы строим сложенный график, но сортируем его по токсичным комментариям, чтобы (возможно) увидеть что-то интересное
toxic_vs_non_toxic.sort_values(by='toxic').plot(kind='bar', stacked=True, figsize=(30,10), fontsize=20).legend(prop={'size': 20})

The diagram above is certainly one way of looking at our data. However, it is missing the complete picture because we are not using two things. First, for each example we have a score (target) of how toxic the comment is. Second, each identity also has a value between 0 to 1 to identify how much they have been targeted. We can use this two aspects to our advantage and see which identities are more frequently related to toxic comments.
Диаграмма выше, безусловно, является одним из способов взглянуть на наши данные. Тем не менее, отсутствует полная картина, потому что мы не используем две вещи. Во-первых, для каждого примера у нас есть оценка (цель) того, насколько ядовитым является комментарий. Во-вторых, каждая идентичность также имеет значение от 0 до 1, чтобы определить, насколько они были нацелены. Мы можем использовать эти два аспекта в наших интересах и посмотреть, какие личности чаще всего связаны с ядовитыми комментариями.

In [ ]:
# First we multiply each identity with the target 
# Сначала мы умножаем каждую идентичность с целью
weighted_toxic = train_labeled_df.iloc[:, 1:].multiply(train_labeled_df.iloc[:, 0], axis="index").sum() 
# changing the value of identity to 1 or 0 only and get comment count per identity group
# изменяя значение идентификатора только на 1 или 0 и получая количество комментариев на группу идентификаторов
identity_label_count = train_labeled_df[identities].where(train_labeled_df == 0, other = 1).sum()
# then we divide the target weighted value by the number of time each identity appears
# затем мы делим целевое взвешенное значение на количество раз, когда появляется каждая идентичность
weighted_toxic = weighted_toxic / identity_label_count
weighted_toxic = weighted_toxic.sort_values(ascending=False)
# plot the data using seaborn like before
# построить данные, используя seaborn, как и раньше
plt.figure(figsize=(30,20))
sns.set(font_scale=3)
ax = sns.barplot(x = weighted_toxic.values , y = weighted_toxic.index, alpha=0.8)
plt.ylabel('Demographics')
plt.xlabel('Weighted Toxicity')
plt.title('Weighted Analysis of Most Frequent Identities')
plt.show()

As we can see from the graph above, the two race based identities (White and Black) and religion based identities (Muslim and Jews) are heavily associated with toxic comments. 
Как видно из приведенного выше графика, две расы (белые и черные) и религии (мусульмане и евреи) тесно связаны с ядовитыми комментариями.

## Section 3: Time Series Analysis Анализ временных рядов
The dataset also has a 'created_date' field, which tells us when the comment was made. We can use this field to do some time series analysis. In this section, we will also use plotly for simplifying our visualization needs.**
В наборе данных также есть поле create_date, которое сообщает нам, когда был сделан комментарий. Мы можем использовать это поле для анализа временных рядов. В этом разделе мы также будем использовать сюжет для упрощения наших потребностей в визуализации. **

In [ ]:
# lets take the dataset with identitiy tags, created date, and target column
# давайте возьмем набор данных с тегами идентификации, датой создания и целевым столбцом
with_date_df = train_df.loc[:, ['created_date', 'target'] + identities].dropna()
# next we will create a weighted dataframe for each identity tag (like we did before)
# first we divide each identity tag with the total value it has in the dataset
# далее мы создадим взвешенный информационный кадр для каждого тега идентичности (как мы делали раньше)
# сначала мы делим каждый тег тождественности на общее значение, которое он имеет в наборе данных
weighted_df = with_date_df.iloc[:, 2:] / with_date_df.iloc[:, 2:].sum()
# then we multiplty this value with the target 
# тогда мы умножаем это значение с целью
target_weighted_df = weighted_df.multiply(with_date_df.iloc[:, 1], axis="index")
# lets add a column to count the number of comments
# позволяет добавить столбец для подсчета количества комментариев
target_weighted_df['comment_count'] = 1
# now we add the date to our newly created dataframe (also parse the text date as datetime)
# теперь мы добавляем дату в наш недавно созданный фрейм данных (также анализируем текстовую дату как datetime)
target_weighted_df['created_date'] = pd.to_datetime(with_date_df['created_date'].apply(lambda dt: dt[:10]))
# now we can do a group by of the created date to count the number of times a identity appears for that date
# теперь мы можем сделать группировку по дате создания, чтобы посчитать, сколько раз личность появляется на эту дату
identity_weight_per_date_df = target_weighted_df.groupby(['created_date']).sum().sort_index()

In [ ]:
# lets group most of the identities into three major categories as follows for simplified analysis
# позволяет сгруппировать большинство идентификаторов в три основные категории следующим образом для упрощенного анализа
races = ['black','white','asian','latino','other_race_or_ethnicity']
religions = ['atheist', 'buddhist', 'christian', 'hindu', 'muslim', 'jewish','other_religion']
sexual_orientation = ['heterosexual', 'homosexual_gay_or_lesbian', 'bisexual', 'other_sexual_orientation']

In [ ]:
# lets create a column to aggregate our weighted toxicity score per identity group
# позволяет создать столбец для агрегирования нашего взвешенного показателя токсичности для каждой идентификационной группы
identity_weight_per_date_df['races_total'] = identity_weight_per_date_df[races].sum(axis=1)
identity_weight_per_date_df['religions_total'] = identity_weight_per_date_df[religions].sum(axis=1)
identity_weight_per_date_df['sexual_orientation_total'] = identity_weight_per_date_df[sexual_orientation].sum(axis=1)
# and then plot a time-series line plot per identity group
# и затем построить линейный график временного ряда для каждой группы идентификаторов
identity_weight_per_date_df[['races_total', 'religions_total', 'sexual_orientation_total']].plot(figsize=(15,7), linewidth=1, fontsize=15) 
plt.legend(loc=2, prop={'size': 15})
plt.xlabel('Comment Date', fontsize=15)
plt.ylabel('Weighted Toxic Score', fontsize=15)

1. At first sight, what a sad graph to look at. First of all, there are many many spikes in our dataset, so we should zoom into each identity per group to see more. Furthermore, we notice a trend of more toxic messages as time goes by. What a depressing sight :( However, we should keep in mind that the distribution of data over the timeline is probably skewed. Lets check it out.[](http://)
1. Какой на первый взгляд грустный график. Прежде всего, в нашем наборе данных много много пиков, поэтому мы должны увеличить каждую идентификацию в каждой группе, чтобы увидеть больше. Кроме того, мы замечаем тенденцию к более ядовитым сообщениям с течением времени. Какое удручающее зрелище :( Однако мы должны помнить, что распределение данных по временной шкале, вероятно, искажено. Давайте проверим это.

In [ ]:
identity_weight_per_date_df['comment_count'].plot(figsize=(15,7), linewidth=1, fontsize=15)
plt.xlabel('Comment Date', fontsize = 15)
plt.ylabel('Total Comments', fontsize = 15)

As we dont have an even distribution of data for our timeline, we can use the comment count to get a better relative weighted toxic score and see where the peaks are.
Поскольку у нас нет равномерного распределения данных для нашей временной шкалы, мы можем использовать счетчик комментариев, чтобы получить лучший относительный взвешенный токсический показатель и посмотреть, где находятся пики.

In [ ]:
# lets divide by the comment count for the date to get a relative weighted toxic score
# давайте разделим на количество комментариев для даты, чтобы получить относительный взвешенный токсический балл
identity_weight_per_date_df['races_rel'] = identity_weight_per_date_df['races_total'] / identity_weight_per_date_df['comment_count']
identity_weight_per_date_df['religions_rel'] = identity_weight_per_date_df['religions_total'] / identity_weight_per_date_df['comment_count']
identity_weight_per_date_df['sexual_orientation_rel'] = identity_weight_per_date_df['sexual_orientation_total']  / identity_weight_per_date_df['comment_count']
# now lets plot the data # теперь позволяет отображать данные
identity_weight_per_date_df[['races_rel', 'religions_rel', 'sexual_orientation_rel']].plot(figsize=(15,7), linewidth=1, fontsize=20) 
plt.legend(loc=2, prop={'size': 15})
plt.xlabel('Comment Date', fontsize=15)
plt.ylabel('Relative Weighted Toxic Score', fontsize=15)

In [ ]:
# lets plot relative weighted toxic score for each identity of races
# позволяет построить относительную взвешенную токсическую оценку для каждой личности рас
identity_weight_per_date_df[races].div(identity_weight_per_date_df['comment_count'], axis=0).plot(figsize=(15,7), linewidth=1, fontsize=15)
plt.legend(loc=2, prop={'size': 15})
plt.xlabel('Comment Date', fontsize=15)
plt.ylabel('Relative Weighted Toxic Score', fontsize=15)

In [ ]:
# lets plot relative weighted toxic score for each identity of religions
# позволяет построить относительный взвешенный токсический балл для каждой религии
identity_weight_per_date_df[religions].div(identity_weight_per_date_df['comment_count'], axis=0).plot(figsize=(15,7), linewidth=1, fontsize=15)
plt.legend(loc=2, prop={'size': 15})
plt.xlabel('Comment Date', fontsize=15)
plt.ylabel('Relative Weighted Toxic Score', fontsize=15)

In [ ]:
# lets plot relative weighted toxic score for each identity of sexual orientation
# позволяет построить относительную взвешенную токсическую оценку для каждой идентичности сексуальной ориентации
identity_weight_per_date_df[sexual_orientation].div(identity_weight_per_date_df['comment_count'], axis=0).plot(figsize=(15,7), linewidth=1, fontsize=20)
plt.legend(loc=2, prop={'size': 15})
plt.xlabel('Comment Date', fontsize=15)
plt.ylabel('Relative Weighted Toxic Score', fontsize=15)

Interesting! After looking at the visualization above, we can conclude the few following things:

* Amount of hate or toxic comments directed toward any of the group isn't constant
* The dataset is full of spikes or relative maximums
* This probably happens due to certain events that triggers hate toward a particular identity
Интересно! Посмотрев на приведенную выше визуализацию, мы можем сделать следующие выводы:

* Количество ненавистных или ядовитых комментариев, адресованных какой-либо группе, не является постоянным
* Набор данных полон пиков или относительных максимумов
* Это, вероятно, происходит из-за определенных событий, которые вызывают ненависть к определенной личности

## Section 4: Extracting the Peaks / Relative Maximums Извлечение пиков / относительных максимумов
As we saw in the last section, since there are so many spikes in our dataset, it is very easy to get distracted. To have a better idea we must zoom out. In this section we do that by extracting the peaks. This could be challenging, as we are not just looking for one global maximum. On the other hand, even if we sort it, we will not be able to find all the relative maximums without writing a complex algorithm to do so.
Как мы видели в последнем разделе, поскольку в нашем наборе данных очень много пиков, очень легко отвлечься. Чтобы иметь лучшую идею, мы должны уменьшить масштаб. В этом разделе мы делаем это путем извлечения пиков. Это может быть сложной задачей, поскольку мы не просто ищем единого глобального максимума. С другой стороны, даже если мы отсортируем это, мы не сможем найти все относительные максимумы без написания сложного алгоритма для этого.

Fortunately, there is a great algorithm from **scipy** library called **'argrelextrema'** that will allow us to extract the relative maximum points from our dataset. For more details about this algorithm, check out the document section: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.argrelextrema.html
К счастью, есть замечательный алгоритм из библиотеки ** scipy **, называемый ** 'argrelextrema' **, который позволит нам извлечь относительные максимальные точки из нашего набора данных. Для более подробной информации об этом алгоритме, посмотрите раздел документа:

In [ ]:
# lets import the algorithm # позволяет импортировать алгоритм
from scipy.signal import argrelextrema

Using this algorithm, we want to create a pandas dataframe to capture all the extreme points in our dataset that captures all the peak points per identity group. We will also want to do a scatter plot in the next section, and scatterplot implementation needs both values (x and y) to be numeric. So we also better create a column to represent the dates in a more numeric form. 

We can do this by taking the date of the first comment as our reference point, and counting how many days the comment was made from the first comment in our dataset. Therefore, in the end we should get a dataframe with the following columns:

| identity | created data | score | days_from_first |
Используя этот алгоритм, мы хотим создать фрейм данных pandas для захвата всех крайних точек в нашем наборе данных, который фиксирует все пиковые точки на группу идентичности. Мы также хотим создать диаграмму рассеяния в следующем разделе, и для реализации диаграммы рассеяния необходимо, чтобы оба значения (x и y) были числовыми. Поэтому мы также лучше создадим столбец для представления дат в более числовой форме.

Мы можем сделать это, взяв дату первого комментария в качестве нашей контрольной точки и посчитав, сколько дней был сделан комментарий с первого комментария в нашем наборе данных. Следовательно, в конце мы должны получить кадр данных со следующими столбцами:

| личность | созданные данные | оценка | дни _ с_первого |

In [ ]:
# we will store all the different datapoints in the following dataframe
# мы будем хранить все различные точки данных в следующем фрейме данных
peaks_df = pd.DataFrame()

# first we loop through all the different identities we are interested in
# сначала мы перебираем все разные личности, которые нас интересуют
for col in races + religions + sexual_orientation:
    # we pass the values through the algorithm to get an index of the rel. maximums 
    # мы передаем значения через алгоритм, чтобы получить индекс отн. максимумы
    _max_index = argrelextrema(identity_weight_per_date_df[col].values, np.greater, order=15)
    # we use the index returned to create a dataframe of the values for those index. in this case 
    # we are interested in the created date and the score, notice how the dataframe needs to be 
    # transformed because of the orientation of the arrays we started off with
    # мы используем возвращаемый индекс для создания кадра данных значений для этого индекса. в этом случае
    # нас интересует дата создания и оценка, обратите внимание, как должен быть датафрейм
    # преобразован из-за ориентации массивов, с которых мы начали
    col_peaks_df = pd.DataFrame(data = [identity_weight_per_date_df.index[_max_index], identity_weight_per_date_df[col].values[_max_index]]).T
    col_peaks_df.columns = ['created_date','score']
    # we create a new column labeling the identity so we can track which peak came from which identity
    # мы создаем новый столбец с меткой идентичности, чтобы мы могли отслеживать, какой пик пришел с какой идентичности
    col_peaks_df['identity'] = col
    # and we keep appending to our main dataframe 
    # и мы продолжаем добавлять к нашему основному фрейму данных
    peaks_df = peaks_df.append(col_peaks_df)
# lets set identity as our index and we are done
# давайте установим идентичность как наш индекс, и мы закончили
peaks_df = peaks_df.set_index('identity')

In [ ]:
# to count the number of days from the first comment, we take our labeled data and convert the created date column
# чтобы посчитать количество дней с момента первого комментария, 
# мы берем наши помеченные данные и конвертируем созданный столбец даты
comments_with_date_df = train_df.loc[:, ['created_date', 'target','comment_text'] + identities].dropna()
comments_with_date_df['created_date'] = pd.to_datetime(with_date_df['created_date'].apply(lambda dt: dt[:10]))
comments_with_date_df['comment_count'] = 1

In [ ]:
# calculate days from first comment
# рассчитать дни с первого комментария
first_dt = min(comments_with_date_df['created_date'].values)
last_dt = max(comments_with_date_df['created_date'].values)
peaks_df['days_from_first'] = (peaks_df['created_date'] - first_dt).dt.days

In [ ]:
# here is a peak at what our peaks_df looks like
# вот вершина того, как выглядит наш peaks_df
peaks_df.head()

## Section 5: Visualizing Relative Maximums Визуализация относительных максимумов

Scatter plot is typically used to show if there is a correlation between two variables. However, in this case, we can also use scatterplot to draw the relative maximums as descrete points. The nice thing about a scatter plot is that we can visualize a third variable as the size of the point. Let's take advantage of this and use the number of comments made as the size of the relative maximum points.
График рассеяния обычно используется, чтобы показать, есть ли корреляция между двумя переменными. Однако в этом случае мы также можем использовать диаграмму рассеяния, чтобы нарисовать относительные максимумы в качестве дискретных точек. Хорошая вещь о точечной диаграмме состоит в том, что мы можем визуализировать третью переменную как размер точки. Давайте воспользуемся этим и используем количество комментариев, сделанных в качестве размера относительных максимальных баллов.

In [ ]:
# lets create a function that returns the peaks dataframe for a given identity
# we also want to get the number of toxic comments made against that identity in the dataframe
# позволяет создать функцию, которая возвращает пиковый фрейм данных для данного идентификатора
# мы также хотим получить количество токсичных комментариев по отношению к этой идентичности в кадре данных
def get_identity_peaks_df(identity, peaks_df, comments_with_date_df):
    # select subset and sort
    # выбрать подмножество и отсортировать
    identity_peaks_df = peaks_df[peaks_df.index==identity].sort_values(by='score', ascending=False)
    # change the score type to float # изменить тип счета на плавающий
    identity_peaks_df['score'] = identity_peaks_df.score.astype(float)
    # use created date as the index so we can join over in later step
    # использовать дату создания в качестве индекса, чтобы мы могли присоединиться на следующем шаге
    identity_peaks_df = identity_peaks_df.set_index('created_date')
    # calculate how many toxic comments were made targetting the given identity group
    # подсчитать, сколько ядовитых комментариев было сделано для данной идентификационной группы
    identity_comment_count_df = comments_with_date_df[comments_with_date_df[identity] > 0][['created_date','comment_count']].groupby('created_date').sum()
    # do an inner join to also get the total number of comments made that day for the given identity
    # сделать внутреннее объединение, чтобы также получить общее количество комментариев, сделанных в этот день для данной личности
    identity_peaks_df = identity_peaks_df.join(identity_comment_count_df)
    return identity_peaks_df

In [ ]:
# to make our scatter plot more presentable we will set the max and min of our y axis
# чтобы сделать наш точный график более презентабельным, мы установим максимальное и минимальное значения нашей оси y
y_lim_min = peaks_df['score'].max() + peaks_df['score'].max() / 3 
# adding a little bit head room on y axis
# добавить немного свободного места на оси у
y_lim_max = peaks_df['score'].min() 

In [ ]:
# now lets write a function that draws the scatter plot for a given identity
# теперь давайте напишем функцию, которая рисует график рассеяния для данного тождества
def identity_scatter_plot(identity, identity_peaks_df, y_lim_min, y_lim_max):
    x = identity_peaks_df['days_from_first'].values
    y = identity_peaks_df['score'].values
    size = identity_peaks_df['comment_count'].values
    label = identity_peaks_df['comment_count'].index
    plt.figure(figsize=(15,7))
    plt.tick_params(axis='both', which='major', labelsize=16)
    plt.scatter(x, y, s=size, label=label)
    plt.ylim(y_lim_max, y_lim_min)
    axis_font = {'fontname':'Arial', 'size':'14'}
    plt.title('Relative Maximums - Targeted Against '+ identity.capitalize() +' Identity', fontsize=15)
    plt.xlabel('Comment Date', fontsize=15)
    plt.ylabel('Relative Weighted Toxic Score', fontsize=15)

Now we are ready to plot each relative maximums per identity group. However, lets only look at top 5 for now. If you remember the horizontal bar plot we did at the start of our tutorial (second graph from the top), you can see that the identities that are most frequently targeted are white, black, homosexual_gay_or_lesbian, muslim, and jewish.
Теперь мы готовы построить каждый относительный максимум для группы идентичности. Тем не менее, давайте пока посмотрим только на топ-5. Если вы помните горизонтальный столбчатый график, который мы сделали в начале нашего урока (второй график сверху), вы можете увидеть, что наиболее часто целевые личности - это белые, черные, homosexual_gay_or_lesbian, мусульманские и еврейские.

In [ ]:
identity = 'white'
identity_peaks_df = get_identity_peaks_df(identity, peaks_df, comments_with_date_df)
identity_scatter_plot(identity, identity_peaks_df, y_lim_min, y_lim_max)

In [ ]:
identity = 'black'
identity_peaks_df = get_identity_peaks_df(identity, peaks_df, comments_with_date_df)
identity_scatter_plot(identity, identity_peaks_df, y_lim_min, y_lim_max)

In [ ]:
identity = 'homosexual_gay_or_lesbian'
identity_peaks_df = get_identity_peaks_df(identity, peaks_df, comments_with_date_df)
identity_scatter_plot(identity, identity_peaks_df, y_lim_min, y_lim_max)

In [ ]:
identity = 'muslim'
identity_peaks_df = get_identity_peaks_df(identity, peaks_df, comments_with_date_df)
identity_scatter_plot(identity, identity_peaks_df, y_lim_min, y_lim_max)

In [ ]:
identity = 'jewish'
identity_peaks_df = get_identity_peaks_df(identity, peaks_df, comments_with_date_df)
identity_scatter_plot(identity, identity_peaks_df, y_lim_min, y_lim_max)

This is great! I will write down the analysis later! Meanwhile, if you have a great observation, write it down in the comment section =) 
Это замечательно! Я запишу анализ позже! Между тем, если у вас есть отличное наблюдение, запишите его в разделе комментариев =)